In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [25]:
import matplotlib.pyplot as plt
import seaborn as sns

#### We are going to import our cleaned dataset, From my Data Preprocessing Project

In [26]:
df = pd.read_csv("/kaggle/input/ames-final-df/AMES_Final_DF.csv")

In [27]:
df.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,...,0,0,0,0,1,0,0,0,1,0
1,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,0,0,0,0,1,0,0,0,1,0
2,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,0,0,0,0,1,0,0,0,1,0
3,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,...,0,0,0,0,1,0,0,0,1,0
4,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,0,0,0,0,1,0,0,0,1,0


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Columns: 274 entries, Lot Frontage to Sale Condition_Partial
dtypes: float64(11), int64(263)
memory usage: 6.1 MB


In [29]:
X = df.drop("SalePrice", axis=1)

In [30]:
y = df["SalePrice"]

In [31]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=50)
# I am holding out 10% of the data to test  the model on a data it's never saw

In [34]:
from sklearn.preprocessing import StandardScaler

In [35]:
scaler = StandardScaler()

In [36]:
scaled_X_train = scaler.fit_transform(X_train)

In [37]:
scaled_X_test = scaler.transform(X_test)
# We are not fitting the the test data because we don't want the model to know the test data

In [39]:
# We are creating an instance of the ElasticNet model, this will help the model in reducing overfitting
# and underfitting on the training data

from sklearn.linear_model import ElasticNet

In [47]:
# Creating an Instance of the ElasticNet model

base_elastic_model = ElasticNet(max_iter=1000000)

In [48]:
param_grid = {'alpha':[0.1,1,5,10,100],'l1_ratio':[.1,.7,.99,1]}

In [49]:
from sklearn.model_selection import GridSearchCV

In [50]:
grid_model = GridSearchCV(base_elastic_model,param_grid=param_grid,scoring='neg_mean_squared_error',
                         cv=5,verbose=1)

In [51]:
grid_model.fit(scaled_X_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(cv=5, estimator=ElasticNet(max_iter=1000000),
             param_grid={'alpha': [0.1, 1, 5, 10, 100],
                         'l1_ratio': [0.1, 0.7, 0.99, 1]},
             scoring='neg_mean_squared_error', verbose=1)

In [52]:
# Let's See the best hyperparameters combination

grid_model.best_estimator_

ElasticNet(alpha=100, l1_ratio=1, max_iter=1000000)

In [53]:
# Then we are to evaluate our model's performance on the 10% per

y_pred = grid_model.predict(scaled_X_test)

In [54]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [56]:
mean_absolute_error(y_test,y_pred)

14726.898588147422

In [57]:
np.sqrt(mean_squared_error(y_test,y_pred))

21690.500001787812

In [58]:
# How do we know if this is actually are good error value for our predictions
np.mean(df['SalePrice'])

180815.53743589742